In [1]:
import pandas as pd
import numpy as np


In [3]:
file_path = "../data/nifty_with_regime.csv"

df = pd.read_csv(file_path, parse_dates=["datetime"])
df.set_index("datetime", inplace=True)

df.head()


,open,high,low,close,volume,ema_5,ema_15,ema_gap,returns,volatility,candle_range,candle_body,hour,minute,day_of_week,returns_lag_1,returns_lag_3,ema_gap_lag_1,regime
datetime,,,,,,,,,,,,,,,,,,,
2015-01-09 11:00:00,8285.75,8288.85,8282.30,8288.00,0,8285.789434,8286.208532,-0.419098,0.000278,0.000489,6.55,2.25,11,0,4,0.000127,0.000060,-1.268458,0
2015-01-09 11:05:00,8287.95,8288.10,8280.20,8281.55,0,8284.376289,8285.626216,-1.249927,-0.000778,0.000485,7.90,6.40,11,5,4,0.000278,0.000338,-0.419098,0
2015-01-09 11:10:00,8281.85,8286.35,8279.90,8282.35,0,8283.700860,8285.216689,-1.515829,0.000097,0.000465,6.45,0.50,11,10,4,-0.000778,0.000127,-1.249927,0
2015-01-09 11:15:00,8282.55,8283.20,8276.15,8278.45,0,8281.950573,8284.370853,-2.420280,-0.000471,0.000457,7.05,4.10,11,15,4,0.000097,0.000278,-1.515829,-1
2015-01-09 11:20:00,8278.55,8280.70,8266.55,8266.55,0,8276.817049,8282.143246,-5.326197,-0.001437,0.000549,14.15,12.00,11,20,4,-0.000471,-0.000778,-2.420280,-1


In [5]:
df["signal"] = 0

# Long signal
df.loc[
    (df["ema_5"] > df["ema_15"]) &
    (df["ema_5"].shift(1) <= df["ema_15"].shift(1)) &
    (df["regime"] == 1),
    "signal"
] = 1

# Short signal
df.loc[
    (df["ema_5"] < df["ema_15"]) &
    (df["ema_5"].shift(1) >= df["ema_15"].shift(1)) &
    (df["regime"] == -1),
    "signal"
] = -1


In [7]:
# Hold position until exit
df["position"] = df["signal"].replace(to_replace=0, method="ffill")

# No position during sideways regime
df.loc[df["regime"] == 0, "position"] = 0

# Shift for next candle execution
df["position"] = df["position"].shift(1)


C:\Users\Hp\AppData\Local\Temp\ipykernel_9912\1652322422.py:2: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df["position"] = df["signal"].replace(to_replace=0, method="ffill")


In [9]:
df["strategy_returns"] = df["position"] * df["returns"]


In [11]:
df["equity_curve"] = (1 + df["strategy_returns"]).cumprod()


In [13]:
def performance_metrics(returns):
    total_return = (1 + returns).prod() - 1
    sharpe = np.sqrt(252 * 78) * returns.mean() / returns.std()
    max_dd = (df["equity_curve"].cummax() - df["equity_curve"]).max()
    
    return total_return, sharpe, max_dd

total_return, sharpe, max_dd = performance_metrics(df["strategy_returns"])

total_return, sharpe, max_dd


(1.2321033437878346, 0.6102765197480765, 0.6555265344720138)

In [15]:
total_trades = df["signal"].abs().sum()
win_rate = (df["strategy_returns"] > 0).mean()

total_trades, win_rate


(3931, 0.41744351315761974)

In [19]:
output_path = "../results/baseline_backtest.csv"
df.to_csv(output_path)

print(f"Baseline backtest saved to {output_path}")


Baseline backtest saved to ../results/baseline_backtest.csv
